In [2]:
from handle_base64 import(
    decode_base64_to_image,
    encode_image_to_base64
)

---
# Prompt to video

In [8]:
#!/usr/bin/env python3
import os
import time
import json
import requests
from datetime import datetime, timezone, timedelta
from typing import Optional


class VeoVideoGenerator:
    def __init__(self, base_url: str = "https://api.thucchien.ai/gemini/v1beta",
                 api_key: str = "sk-YzYUqhbM9WskmD2AQDUXJQ"):
        self.base_url = base_url
        self.api_key = api_key
        self.headers = {
            "x-goog-api-key": api_key,
            "Content-Type": "application/json"
        }

        # --- Tạo thư mục lưu kết quả ---
        os.makedirs("results_video", exist_ok=True)
        os.makedirs("logs", exist_ok=True)

        # --- File log ---
        self.LOG_FILE = "logs/full_log.jsonl"
        self.LOG_FILE_SUMMARY = "logs/summary.log"

        # --- Múi giờ Hà Nội ---
        self.hanoi_tz = timezone(timedelta(hours=7))

    # ========== Core Veo API Calls ==========
    
    def generate_video(self, prompt: str, input_image_path: str | None) -> Optional[str]:
        """
        Initiate video generation with Veo.

        Args:
            prompt: Text description of the video to generate
            
        Returns:
            Operation name if successful, None otherwise
        """
        print(f"🎬 Generating video with prompt: '{prompt}'")

        url = f"{self.base_url}/models/veo-3.0-generate-001:predictLongRunning"

        payload = {
            "instances": [{
                "prompt": prompt,
            }],

            "parameters": {
                "negativePrompt": "",
                "aspectRatio": "16:9", 
                "durationSeconds": 8, # Thời gian của video (tính bằng giây), từ 5-8. Chỉ hỗ trợ cho Veo 2. Mặc định là 8 giây.
                "resolution" : "1080p",
                "personGeneration" : "allow_all" #Text-to-video: Chỉ hỗ trợ "allow_all". Image-to-video: Chỉ hỗ trợ "allow_adult".
            }

        }

        try:
            response = requests.post(url, headers=self.headers, json=payload)
            response.raise_for_status()
            
            data = response.json()
            operation_name = data.get("name")
            
            if operation_name:
                print(f"✅ Video generation started: {operation_name}")
                return operation_name
            else:
                print("❌ No operation name returned")
                print(f"Response: {json.dumps(data, indent=2)}")
                return None
                
        except requests.RequestException as e:
            print(f"❌ Failed to start video generation: {e}")
            if hasattr(e, 'response') and e.response is not None:
                try:
                    error_data = e.response.json()
                    print(f"Error details: {json.dumps(error_data, indent=2)}")
                except:
                    print(f"Error response: {e.response.text}")
            return None

    def wait_for_completion(self, operation_name: str, max_wait_time: int = 600) -> Optional[str]:
        print("⏳ Waiting for video generation to complete...")

        operation_url = f"{self.base_url}/{operation_name}"
        start_time = time.time()
        poll_interval = 10

        while time.time() - start_time < max_wait_time:
            try:
                response = requests.get(operation_url, headers=self.headers)
                response.raise_for_status()
                data = response.json()

                if "error" in data:
                    print("❌ Error in video generation:")
                    print(json.dumps(data["error"], indent=2))
                    return None

                if data.get("done"):
                    print("🎉 Video generation complete!")
                    try:
                        video_uri = data["response"]["generateVideoResponse"]["generatedSamples"][0]["video"]["uri"]
                        print(f"📹 Video URI: {video_uri}")
                        return video_uri
                    except KeyError:
                        print("❌ Could not extract video URI")
                        print(json.dumps(data, indent=2))
                        return None

                print(f"🔍 Still processing... ({int(time.time() - start_time)}s)")
                time.sleep(poll_interval)
                poll_interval = min(poll_interval * 1.2, 30)

            except requests.RequestException as e:
                print(f"❌ Polling error: {e}")
                time.sleep(poll_interval)

        print("⏰ Timeout")
        return None

    def download_video(self, video_uri: str, output_filename: str) -> Optional[str]:
        print(f"⬇️ Downloading video...")
        print(f"Original URI: {video_uri}")

        if video_uri.startswith("https://generativelanguage.googleapis.com/"):
            relative_path = video_uri.replace(
                "https://generativelanguage.googleapis.com/", "")
        else:
            relative_path = video_uri

        base_path = self.base_url.replace("/v1beta", "/download")
        download_url = f"{base_path}/{relative_path}"

        try:
            response = requests.get(download_url, headers=self.headers, stream=True, allow_redirects=True)
            response.raise_for_status()

            with open(output_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

            if os.path.getsize(output_filename) > 0:
                print(f"✅ Video saved as: {output_filename}")
                return output_filename
            else:
                print("❌ Empty file, deleting...")
                os.remove(output_filename)
                return None

        except requests.RequestException as e:
            print(f"❌ Download failed: {e}")
            return None

    # ========== Logging Helpers ==========
    def log_results(self, prompt: str, model: str, video_path: Optional[str], full_response: dict):
        now = datetime.now(self.hanoi_tz)
        timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

        log_entry = {
            "timestamp": timestamp,
            "prompt": prompt,
            "model": model,
            "video_path": video_path,
            "full_response": full_response
        }

        # --- Log chi tiết (JSONL) ---
        with open(self.LOG_FILE, "a", encoding="utf-8") as f:
            f.write(json.dumps(log_entry, ensure_ascii=False))
            f.write("\n")

        # --- Log tóm tắt (đẹp kiểu ChatGPT) ---
        with open(self.LOG_FILE_SUMMARY, "a", encoding="utf-8") as f:
            f.write(f"\n=================== 🕒 {timestamp} (UTC+7) ===================\n\n")
            f.write("🎬 Prompt:\n")
            f.write(prompt + "\n\n")
            f.write(f"📊 Model: {model}\n")
            if video_path:
                f.write("🎥 Video đã lưu:\n")
                f.write(f"   → {video_path}\n\n")
            else:
                f.write("⚠️ Video chưa được lưu hoặc lỗi khi tải xuống.\n\n")
            f.write("======================================================================\n")

        print(f"📝 Log saved to {self.LOG_FILE} and {self.LOG_FILE_SUMMARY}")

    # ========== Full Workflow ==========
    def generate_and_download(self, prompt: str, input_image_path: str|None):
        now = datetime.now(self.hanoi_tz)
        timestamp_safe = now.strftime("%Y%m%d_%H%M%S")
        video_filename = f"video_{timestamp_safe}.mp4"
        output_path = os.path.join("results_video", video_filename)

        print("=" * 60)
        print("🎬 VEO VIDEO GENERATION WORKFLOW")
        print("=" * 60)

        operation_name = self.generate_video(prompt, input_image_path)
        if not operation_name:
            self.log_results(prompt, "veo-3.0-generate-preview", None, {"error": "Failed to start generation"})
            return False

        video_uri = self.wait_for_completion(operation_name)
        if not video_uri:
            self.log_results(prompt, "veo-3.0-generate-preview", None, {"error": "Failed to complete"})
            return False

        saved_path = self.download_video(video_uri, output_path)
        self.log_results(prompt, "veo-3.0-generate-preview", saved_path, {"video_uri": video_uri})
        return True


def main():
    base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/gemini/v1beta")
    api_key = os.getenv("LITELLM_API_KEY", "sk-YzYUqhbM9WskmD2AQDUXJQ")

    generator = VeoVideoGenerator(base_url=base_url, api_key=api_key)

    # INPUT
    input_image_path="C:/Users/Admin/Desktop/Codice_Vincente/notebook/video/inputs/image-Photoroom222-(1).png"
    prompt = "{\"title\":\"Lời Thề Kháng Chiến – Cảnh Kêu Gọi Hành Động\",\"duration\":\"8s\",\"camera\":{\"style\":\"ống kính góc rộng điện ảnh (35mm), độ sâu trường ảnh lớn, góc quay thấp tạo cảm giác hùng tráng\",\"movement\":\"chuyển động dolly chậm tiến về phía nhân vật, camera ổn định\",\"frame_rate\":24,\"aspect_ratio\":\"16:9\"},\"lighting\":{\"type\":\"ánh sáng chiều muộn / không khí khói lửa\",\"color_temperature\":\"tương phản mạnh giữa cam ấm (7000K) và khói lam/xám lạnh\",\"direction\":\"ánh sáng xiên mạnh từ một bên, làm nổi bật gương mặt kiên nghị\",\"mood\":\"hùng tráng, quyết tâm, mãnh liệt\"},\"environment\":{\"location\":\"khu rừng Việt Nam / khung cảnh chiến trường hoang tàn (thời kỳ kháng chiến)\",\"weather\":\"khói mù, bụi trong không khí\",\"details\":\"hầm hào bị phá hủy, đất đá văng tung, rừng rậm bao quanh, lá cờ Tổ quốc mờ ảo ở hậu cảnh (độ mờ thấp, mang tính biểu tượng)\"},\"character\":{\"gender\":\"nam\",\"age\":\"khoảng 20 tuổi\",\"appearance\":\"người lính Việt Nam, mặc quân phục kaki xanh giản dị, đội mũ cối hơi nghiêng, gương mặt cương nghị\",\"emotion\":\"quyết tâm mãnh liệt, truyền cảm hứng, hướng về tương lai\",\"action\":[{\"time\":\"0–2s\",\"description\":\"người lính đứng vững giữa khung cảnh khói lửa, nhìn thẳng về phía trước với ánh mắt kiên định\"},{\"time\":\"2–4s\",\"description\":\"anh giơ nắm tay phải lên cao trước ngực hoặc chỉ tay về phía trước, như đang hô vang lời thề: \"Quyết tử cho tổ quốc quyết sinh\" \"},{\"time\":\"4–6s\",\"description\":\"cảnh quay cận gương mặt, ánh sáng hắt ngang làm nổi rõ ánh mắt và đường nét kiên cường\"},{\"time\":\"6–8s\",\"description\":\"camera tiếp tục tiến gần, khói dần tan, để lộ rõ ánh nhìn quyết liệt và tinh thần thép của người lính\"}]},\"visual_style\":{\"color_palette\":[\"#CC3300\",\"#FF9900\",\"#4A6E7D\",\"#1C1C1C\"],\"contrast\":\"độ tương phản cao, phong cách chiaroscuro (ánh sáng – bóng tối rõ nét)\",\"tone\":\"chất phim điện ảnh, khói mù dày đặc, ánh sáng nhấn mạnh tông đỏ – vàng tượng trưng cho lòng yêu nước\"},\"sound_reference\":{\"tempo\":\"120 BPM\",\"genre\":\"Nhạc giao hưởng hùng tráng / Trống hành quân\",\"mood_sync\":\"dàn dây trỗi dậy, trống mạnh dồn dập, tiếng gió và tiếng hô xa xa vang vọng\"},\"output\":{\"resolution\":\"1920x1080\",\"format\":\"mp4\",\"length_seconds\":8,\"fps\":24},\"notes\":\"Không chèn chữ, không phụ đề, không logo. Cảnh phim thuần hình ảnh, nhấn mạnh tinh thần anh hùng, khí thế dân tộc và ý chí kiên định của người Việt Nam.\"}"
    success = generator.generate_and_download(prompt, input_image_path)

    if success:
        print("✅ Example completed successfully!")
    else:
        print("❌ Example failed!")


if __name__ == "__main__":
    main()


🎬 VEO VIDEO GENERATION WORKFLOW
🎬 Generating video with prompt: '{"title":"Lời Thề Kháng Chiến – Cảnh Kêu Gọi Hành Động","duration":"8s","camera":{"style":"ống kính góc rộng điện ảnh (35mm), độ sâu trường ảnh lớn, góc quay thấp tạo cảm giác hùng tráng","movement":"chuyển động dolly chậm tiến về phía nhân vật, camera ổn định","frame_rate":24,"aspect_ratio":"16:9"},"lighting":{"type":"ánh sáng chiều muộn / không khí khói lửa","color_temperature":"tương phản mạnh giữa cam ấm (7000K) và khói lam/xám lạnh","direction":"ánh sáng xiên mạnh từ một bên, làm nổi bật gương mặt kiên nghị","mood":"hùng tráng, quyết tâm, mãnh liệt"},"environment":{"location":"khu rừng Việt Nam / khung cảnh chiến trường hoang tàn (thời kỳ kháng chiến)","weather":"khói mù, bụi trong không khí","details":"hầm hào bị phá hủy, đất đá văng tung, rừng rậm bao quanh, lá cờ Tổ quốc mờ ảo ở hậu cảnh (độ mờ thấp, mang tính biểu tượng)"},"character":{"gender":"nam","age":"khoảng 20 tuổi","appearance":"người lính Việt Nam, mặc q

---
# Promt + Image = Video

In [9]:
#!/usr/bin/env python3
import os
import time
import json
import requests
from datetime import datetime, timezone, timedelta
from typing import Optional


class VeoVideoGenerator:
    def __init__(self, base_url: str = "https://api.thucchien.ai/gemini/v1beta",
                 api_key: str = "sk-YzYUqhbM9WskmD2AQDUXJQ"):
        self.base_url = base_url
        self.api_key = api_key
        self.headers = {
            "x-goog-api-key": api_key,
            "Content-Type": "application/json"
        }

        # --- Tạo thư mục lưu kết quả ---
        os.makedirs("results_video", exist_ok=True)
        os.makedirs("logs", exist_ok=True)

        # --- File log ---
        self.LOG_FILE = "logs/full_log.jsonl"
        self.LOG_FILE_SUMMARY = "logs/summary.log"

        # --- Múi giờ Hà Nội ---
        self.hanoi_tz = timezone(timedelta(hours=7))

    # ========== Core Veo API Calls ==========
    
    def generate_video(self, prompt: str, input_image_path: str | None) -> Optional[str]:
        """
        Initiate video generation with Veo.

        Args:
            prompt: Text description of the video to generate
            
        Returns:
            Operation name if successful, None otherwise
        """
        print(f"🎬 Generating video with prompt: '{prompt}'")

        url = f"{self.base_url}/models/veo-3.0-generate-001:predictLongRunning"
        if input_image_path != None:
            base64_encode = encode_image_to_base64(input_image_path)
            mimiType = "image/png"
        else:
            base64_encode= None
            mimiType= None
        payload = {
            "instances": [{
                "prompt": prompt,
                "image": {
                    "bytesBase64Encoded": base64_encode,
                    "mimeType": mimiType
                }
            }],

            "parameters": {
                "negativePrompt": "",
                "aspectRatio": "16:9", 
                "durationSeconds": 8, # Thời gian của video (tính bằng giây), từ 5-8. Chỉ hỗ trợ cho Veo 2. Mặc định là 8 giây.
                "resolution" : "1080p",
                "personGeneration" : "allow_adult" #Text-to-video: Chỉ hỗ trợ "allow_all". Image-to-video: Chỉ hỗ trợ "allow_adult".
            }

        }

        try:
            response = requests.post(url, headers=self.headers, json=payload)
            response.raise_for_status()
            
            data = response.json()
            operation_name = data.get("name")
            
            if operation_name:
                print(f"✅ Video generation started: {operation_name}")
                return operation_name
            else:
                print("❌ No operation name returned")
                print(f"Response: {json.dumps(data, indent=2)}")
                return None
                
        except requests.RequestException as e:
            print(f"❌ Failed to start video generation: {e}")
            if hasattr(e, 'response') and e.response is not None:
                try:
                    error_data = e.response.json()
                    print(f"Error details: {json.dumps(error_data, indent=2)}")
                except:
                    print(f"Error response: {e.response.text}")
            return None

    def wait_for_completion(self, operation_name: str, max_wait_time: int = 600) -> Optional[str]:
        print("⏳ Waiting for video generation to complete...")

        operation_url = f"{self.base_url}/{operation_name}"
        start_time = time.time()
        poll_interval = 10

        while time.time() - start_time < max_wait_time:
            try:
                response = requests.get(operation_url, headers=self.headers)
                response.raise_for_status()
                data = response.json()

                if "error" in data:
                    print("❌ Error in video generation:")
                    print(json.dumps(data["error"], indent=2))
                    return None

                if data.get("done"):
                    print("🎉 Video generation complete!")
                    try:
                        video_uri = data["response"]["generateVideoResponse"]["generatedSamples"][0]["video"]["uri"]
                        print(f"📹 Video URI: {video_uri}")
                        return video_uri
                    except KeyError:
                        print("❌ Could not extract video URI")
                        print(json.dumps(data, indent=2))
                        return None

                print(f"🔍 Still processing... ({int(time.time() - start_time)}s)")
                time.sleep(poll_interval)
                poll_interval = min(poll_interval * 1.2, 30)

            except requests.RequestException as e:
                print(f"❌ Polling error: {e}")
                time.sleep(poll_interval)

        print("⏰ Timeout")
        return None

    def download_video(self, video_uri: str, output_filename: str) -> Optional[str]:
        print(f"⬇️ Downloading video...")
        print(f"Original URI: {video_uri}")

        if video_uri.startswith("https://generativelanguage.googleapis.com/"):
            relative_path = video_uri.replace(
                "https://generativelanguage.googleapis.com/", "")
        else:
            relative_path = video_uri

        base_path = self.base_url.replace("/v1beta", "/download")
        download_url = f"{base_path}/{relative_path}"

        try:
            response = requests.get(download_url, headers=self.headers, stream=True, allow_redirects=True)
            response.raise_for_status()

            with open(output_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

            if os.path.getsize(output_filename) > 0:
                print(f"✅ Video saved as: {output_filename}")
                return output_filename
            else:
                print("❌ Empty file, deleting...")
                os.remove(output_filename)
                return None

        except requests.RequestException as e:
            print(f"❌ Download failed: {e}")
            return None

    # ========== Logging Helpers ==========
    def log_results(self, prompt: str, model: str, video_path: Optional[str], full_response: dict):
        now = datetime.now(self.hanoi_tz)
        timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

        log_entry = {
            "timestamp": timestamp,
            "prompt": prompt,
            "model": model,
            "video_path": video_path,
            "full_response": full_response
        }

        # --- Log chi tiết (JSONL) ---
        with open(self.LOG_FILE, "a", encoding="utf-8") as f:
            f.write(json.dumps(log_entry, ensure_ascii=False))
            f.write("\n")

        # --- Log tóm tắt (đẹp kiểu ChatGPT) ---
        with open(self.LOG_FILE_SUMMARY, "a", encoding="utf-8") as f:
            f.write(f"\n=================== 🕒 {timestamp} (UTC+7) ===================\n\n")
            f.write("🎬 Prompt:\n")
            f.write(prompt + "\n\n")
            f.write(f"📊 Model: {model}\n")
            if video_path:
                f.write("🎥 Video đã lưu:\n")
                f.write(f"   → {video_path}\n\n")
            else:
                f.write("⚠️ Video chưa được lưu hoặc lỗi khi tải xuống.\n\n")
            f.write("======================================================================\n")

        print(f"📝 Log saved to {self.LOG_FILE} and {self.LOG_FILE_SUMMARY}")

    # ========== Full Workflow ==========
    def generate_and_download(self, prompt: str, input_image_path: str|None):
        now = datetime.now(self.hanoi_tz)
        timestamp_safe = now.strftime("%Y%m%d_%H%M%S")
        video_filename = f"video_{timestamp_safe}.mp4"
        output_path = os.path.join("results_video", video_filename)

        print("=" * 60)
        print("🎬 VEO VIDEO GENERATION WORKFLOW")
        print("=" * 60)

        operation_name = self.generate_video(prompt, input_image_path)
        if not operation_name:
            self.log_results(prompt, "veo-3.0-generate-preview", None, {"error": "Failed to start generation"})
            return False

        video_uri = self.wait_for_completion(operation_name)
        if not video_uri:
            self.log_results(prompt, "veo-3.0-generate-preview", None, {"error": "Failed to complete"})
            return False

        saved_path = self.download_video(video_uri, output_path)
        self.log_results(prompt, "veo-3.0-generate-preview", saved_path, {"video_uri": video_uri})
        return True


def main():
    base_url = os.getenv("LITELLM_BASE_URL", "https://api.thucchien.ai/gemini/v1beta")
    api_key = os.getenv("LITELLM_API_KEY", "sk-YzYUqhbM9WskmD2AQDUXJQ")

    generator = VeoVideoGenerator(base_url=base_url, api_key=api_key)

    # INPUT
    input_image_path="C:/Users/Admin/Desktop/Codice_Vincente/notebook/video/inputs/bodoivietnam.jpeg"
    prompt = "{\"title\":\"Lời Thề Kháng Chiến – Cảnh Kêu Gọi Hành Động\",\"duration\":\"8s\",\"camera\":{\"style\":\"ống kính góc rộng điện ảnh (35mm), độ sâu trường ảnh lớn, góc quay thấp tạo cảm giác hùng tráng\",\"movement\":\"chuyển động dolly chậm tiến về phía nhân vật, camera ổn định\",\"frame_rate\":24,\"aspect_ratio\":\"16:9\"},\"lighting\":{\"type\":\"ánh sáng chiều muộn / không khí khói lửa\",\"color_temperature\":\"tương phản mạnh giữa cam ấm (7000K) và khói lam/xám lạnh\",\"direction\":\"ánh sáng xiên mạnh từ một bên, làm nổi bật gương mặt kiên nghị\",\"mood\":\"hùng tráng, quyết tâm, mãnh liệt\"},\"environment\":{\"location\":\"khu rừng Việt Nam / khung cảnh chiến trường hoang tàn (thời kỳ kháng chiến)\",\"weather\":\"khói mù, bụi trong không khí\",\"details\":\"hầm hào bị phá hủy, đất đá văng tung, rừng rậm bao quanh, lá cờ Tổ quốc mờ ảo ở hậu cảnh (độ mờ thấp, mang tính biểu tượng)\"},\"character\":{\"gender\":\"nam\",\"age\":\"khoảng 20 tuổi\",\"appearance\":\"người lính Việt Nam, mặc quân phục kaki xanh giản dị, đội mũ cối hơi nghiêng, gương mặt cương nghị\",\"emotion\":\"quyết tâm mãnh liệt, truyền cảm hứng, hướng về tương lai\",\"action\":[{\"time\":\"0–2s\",\"description\":\"người lính đứng vững giữa khung cảnh khói lửa, nhìn thẳng về phía trước với ánh mắt kiên định\"},{\"time\":\"2–4s\",\"description\":\"anh giơ nắm tay phải lên cao trước ngực hoặc chỉ tay về phía trước, như đang hô vang lời thề: \"Quyết tử cho tổ quốc quyết sinh\" \"},{\"time\":\"4–6s\",\"description\":\"cảnh quay cận gương mặt, ánh sáng hắt ngang làm nổi rõ ánh mắt và đường nét kiên cường\"},{\"time\":\"6–8s\",\"description\":\"camera tiếp tục tiến gần, khói dần tan, để lộ rõ ánh nhìn quyết liệt và tinh thần thép của người lính\"}]},\"visual_style\":{\"color_palette\":[\"#CC3300\",\"#FF9900\",\"#4A6E7D\",\"#1C1C1C\"],\"contrast\":\"độ tương phản cao, phong cách chiaroscuro (ánh sáng – bóng tối rõ nét)\",\"tone\":\"chất phim điện ảnh, khói mù dày đặc, ánh sáng nhấn mạnh tông đỏ – vàng tượng trưng cho lòng yêu nước\"},\"sound_reference\":{\"tempo\":\"120 BPM\",\"genre\":\"Nhạc giao hưởng hùng tráng / Trống hành quân\",\"mood_sync\":\"dàn dây trỗi dậy, trống mạnh dồn dập, tiếng gió và tiếng hô xa xa vang vọng\"},\"output\":{\"resolution\":\"1920x1080\",\"format\":\"mp4\",\"length_seconds\":8,\"fps\":24},\"notes\":\"Không chèn chữ, không phụ đề, không logo. Cảnh phim thuần hình ảnh, nhấn mạnh tinh thần anh hùng, khí thế dân tộc và ý chí kiên định của người Việt Nam.\"}"
    success = generator.generate_and_download(prompt, input_image_path)

    if success:
        print("✅ Example completed successfully!")
    else:
        print("❌ Example failed!")


if __name__ == "__main__":
    main()


🎬 VEO VIDEO GENERATION WORKFLOW
🎬 Generating video with prompt: '{"title":"Lời Thề Kháng Chiến – Cảnh Kêu Gọi Hành Động","duration":"8s","camera":{"style":"ống kính góc rộng điện ảnh (35mm), độ sâu trường ảnh lớn, góc quay thấp tạo cảm giác hùng tráng","movement":"chuyển động dolly chậm tiến về phía nhân vật, camera ổn định","frame_rate":24,"aspect_ratio":"16:9"},"lighting":{"type":"ánh sáng chiều muộn / không khí khói lửa","color_temperature":"tương phản mạnh giữa cam ấm (7000K) và khói lam/xám lạnh","direction":"ánh sáng xiên mạnh từ một bên, làm nổi bật gương mặt kiên nghị","mood":"hùng tráng, quyết tâm, mãnh liệt"},"environment":{"location":"khu rừng Việt Nam / khung cảnh chiến trường hoang tàn (thời kỳ kháng chiến)","weather":"khói mù, bụi trong không khí","details":"hầm hào bị phá hủy, đất đá văng tung, rừng rậm bao quanh, lá cờ Tổ quốc mờ ảo ở hậu cảnh (độ mờ thấp, mang tính biểu tượng)"},"character":{"gender":"nam","age":"khoảng 20 tuổi","appearance":"người lính Việt Nam, mặc q